In [3]:
import pandas as pd
import numpy as np

### DATA STRUCTURE
During Wrangling step of data process, you must also inspect your dataset for two things:
- Data quality issues (dirty data)
    - .head()
    - .tail()
    - .sample()
    - .info()
    - .describe()
    - .value_counts()
    - .isnull()
    - sum(df["col"].isnull())
    - .duplicated()
    - .sort_values()
- Lack of tidiness (messy data)
    - .head()

Assessing is the precursos to cleaning. It's where you identify and categorize common data quality and tidiness issues.

**Low Quality data**  is dirty data, i.e., data with content issues. Examples: Duplicates, missing data, corrupted data

**Unitidy data** is messy data, i.e., data with structural issues.

In tidy data:
1. Each variable forms a column.
2. Each observation forms a row.
3. Each type of observational unit forms a table.

OBS: Always follow the detect -> document -> clean process.

Reference to Tidy Data: https://vita.had.co.nz/papers/tidy-data.pdf

Reference to Reshaping: https://pandas.pydata.org/docs/user_guide/reshaping.html

### Loading Dataframes

In [4]:
# Example of untidy data
untidy_survey = pd.DataFrame({"state": ["MG", "PE", "RJ"], "female": [30, 35, 90], "male": [np.nan, 85, 30]})
untidy_survey

,state,female,male
0,MG,30,NaN
1,PE,35,85.0
2,RJ,90,30.0


In [5]:
# Example of tidy data
tidy_survey = pd.DataFrame({"state": ["MG", "PE", "PE", "RJ", "RJ"], "gender": ["female", "male", "female", "male", "female"], "perc": [30, 85, 35, 30, 90]})
tidy_survey

,state,gender,perc
0,MG,female,30
1,PE,male,85
2,PE,female,35
3,RJ,male,30
4,RJ,female,90


In [9]:
# Joining dataframes

patient_info = pd.DataFrame({"patient_id": [101, 102, 103, 104], "name": ["Italo", "Bruna", "Molier", "Mauraia"], "age": [89, 90, 91, 90]})
patient_dose = pd.DataFrame({"patient_id": [100, 101, 101, 102, 103], "drug": ["frontal", "gardenal", "rivotril", "lexotan", "lexapro"], "dose": [80., 60., 40., 80., 50.]})

## Wrangling Data

In [19]:
patient_dose.rename(columns={"patient_id":"id"})

,id,drug,dose
0,100,frontal,80.0
1,101,gardenal,60.0
2,101,rivotril,40.0
3,102,lexotan,80.0
4,103,lexapro,50.0


In [15]:
# Joining patient_dfs dataframes with .merge
patient_info.merge(patient_dose, how="outer", on="patient_id")

,patient_id,name,age,drug,dose
0,101,Italo,89.0,gardenal,60.0
1,101,Italo,89.0,rivotril,40.0
2,102,Bruna,90.0,lexotan,80.0
3,103,Molier,91.0,lexapro,50.0
4,104,Mauraia,90.0,NaN,NaN
5,100,NaN,NaN,frontal,80.0


In [22]:
# And what about .join?
patient_info_as_index = patient_info.set_index("patient_id")
patient_dose_as_index = patient_dose.set_index("patient_id")

In [23]:
patient_info_as_index

,name,age
patient_id,,
101,Italo,89
102,Bruna,90
103,Molier,91
104,Mauraia,90


In [24]:
patient_dose_as_index

,drug,dose
patient_id,,
100,frontal,80.0
101,gardenal,60.0
101,rivotril,40.0
102,lexotan,80.0
103,lexapro,50.0


In [25]:
patient_info_as_index.join(patient_dose_as_index, how="inner")

,name,age,drug,dose
patient_id,,,,
101,Italo,89,gardenal,60.0
101,Italo,89,rivotril,40.0
102,Bruna,90,lexotan,80.0
103,Molier,91,lexapro,50.0


In [26]:
tidy_survey

,state,gender,perc
0,MG,female,30
1,PE,male,85
2,PE,female,35
3,RJ,male,30
4,RJ,female,90


In [32]:
# Pivot Table: Tidy to Untidy
pd.pivot_table(tidy_survey, index=["state"], values=["perc"], columns="gender")

perc      
gender female  male
state              
MG       30.0   NaN
PE       35.0  85.0
RJ       90.0  30.0

In [33]:
# Melt: Untidy to Tidy
ppl_desc = pd.DataFrame({'first': ['John', 'Mary'], 'last': ['Doe', 'Bo'], 'height': [5.5, 6.0], 'weight': [130, 150]})
ppl_desc

,first,last,height,weight
0,John,Doe,5.5,130
1,Mary,Bo,6.0,150


In [34]:
ppl_desc.melt(id_vars=['first', 'last'], var_name='atributo_corporal')

,first,last,atributo_corporal,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [71]:
# Concatenando dataframes - gerando dataframes
patient_info1 = patient_info
patient_info2 = pd.DataFrame({"patient_id": [105, 106, 107, 108], "name": ["Joao", "Maria", "Bruno", "Betânia"], "age": [12, 10, 10, 23]})

In [72]:
# Apresentando dataframe 1
patient_info1

,patient_id,name,age
0,101,Italo,89
1,102,Bruna,90
2,103,Molier,91
3,104,Mauraia,90


In [74]:
# Apresentando dataframe 2
patient_info2

,patient_id,name,age
0,105,Joao,12
1,106,Maria,10
2,107,Bruno,10
3,108,Betânia,23


In [77]:
# Concatenando dataframes
pd.concat([patient_info1, patient_info2]).reset_index(drop=True)

,patient_id,name,age
0,101,Italo,89
1,102,Bruna,90
2,103,Molier,91
3,104,Mauraia,90
4,105,Joao,12
5,106,Maria,10
6,107,Bruno,10
7,108,Betânia,23


## Issues found

#### Quality Issues
- Indice 5 tem salário muito lato

#### Tidiness Issues
- Colunas 1800:1900 deveriam ser uma única coluna

### Clean

**Issue**: Indice 5 tem salário muito lato

**Soluçao**: Deletar registro do indice 5

In [80]:
#df.drop(index=5, axis=0, inplace=True)

In [ ]:
# Testing


**Issue**: Colunas 1800:1900 deveriam ser uma única coluna

**Soluçao**: Aplicar pd.melt no dataframe

In [81]:
# Aplica melt

In [82]:
# Test
#.head()

In [ ]:
df_clean.to_csv("dataframe_limpo_para_ser_analisado.csv", sep="|", index=True)

#### REAL DATA

In [6]:
folder = "world_data/"

In [7]:
pib = pd.read_csv(folder + "income_per_person_gdppercapita_ppp_inflation_adjusted.csv", sep=",")
lexp = pd.read_csv(folder + "life_expectancy_years.csv", sep=",")
murd = pd.read_csv(folder + "murder_total_deaths.csv", sep=",")
pop = pd.read_csv(folder + "population_total.csv")

In [36]:
pib.head()

,country,1800,1801,1802,1803,1804,1805,1806,1807,1808,...,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040
0,Afghanistan,603,603,603,603,603,603,603,603,603,...,2550,2600,2660,2710,2770,2820,2880,2940,3000,3060
1,Albania,667,667,667,667,667,668,668,668,668,...,19400,19800,20200,20600,21000,21500,21900,22300,22800,23300
2,Algeria,715,716,717,718,719,720,721,722,723,...,14300,14600,14900,15200,15500,15800,16100,16500,16800,17100
3,Andorra,1200,1200,1200,1200,1210,1210,1210,1210,1220,...,73600,75100,76700,78300,79900,81500,83100,84800,86500,88300
4,Angola,618,620,623,626,628,631,634,637,640,...,6110,6230,6350,6480,6610,6750,6880,7020,7170,7310


In [37]:
lexp.head()

,country,1800,1801,1802,1803,1804,1805,1806,1807,1808,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
0,Afghanistan,28.2,28.2,28.2,28.2,28.2,28.2,28.1,28.1,28.1,...,76.5,76.6,76.7,76.9,77.0,77.1,77.3,77.4,77.5,77.7
1,Albania,35.4,35.4,35.4,35.4,35.4,35.4,35.4,35.4,35.4,...,87.4,87.5,87.6,87.7,87.8,87.9,88.0,88.1,88.2,88.3
2,Algeria,28.8,28.8,28.8,28.8,28.8,28.8,28.8,28.8,28.8,...,88.3,88.4,88.5,88.6,88.7,88.8,88.9,89.0,89.1,89.2
3,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,...,78.7,78.9,79.0,79.1,79.3,79.4,79.5,79.7,79.8,79.9


In [38]:
murd.head()

,country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Afghanistan,2070.00,2200.00,2380.00,2600.00,2830.00,3020.00,3160.00,3270.0,3350.00,...,4910.00,4960.00,4990.00,4940.00,5020.00,5190.00,5560.00,5820.00,6060.00,6270.00
1,Albania,160.00,182.00,201.00,221.00,239.00,267.00,295.00,327.0,338.00,...,82.70,77.50,67.50,68.40,68.50,68.50,68.70,68.90,69.20,69.50
2,Algeria,377.00,382.00,391.00,400.00,410.00,421.00,432.00,452.0,468.00,...,437.00,441.00,445.00,447.00,451.00,457.00,465.00,474.00,472.00,471.00
3,Andorra,0.48,0.51,0.54,0.55,0.55,0.53,0.51,0.5,0.49,...,0.52,0.52,0.53,0.54,0.54,0.54,0.55,0.55,0.55,0.55
4,Angola,527.00,532.00,543.00,569.00,598.00,608.00,582.00,582.0,667.00,...,904.00,933.00,958.00,978.00,990.00,1010.00,1030.00,1050.00,1080.00,1090.00


In [39]:
pop.head()

,country,1800,1801,1802,1803,1804,1805,1806,1807,1808,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
0,Afghanistan,3280000,3280000,3280000,3280000,3280000,3280000,3280000,3280000,3280000,...,76600000,76400000,76300000,76100000,76000000,75800000,75600000,75400000,75200000,74900000
1,Albania,400000,402000,404000,405000,407000,409000,411000,413000,414000,...,1330000,1300000,1270000,1250000,1220000,1190000,1170000,1140000,1110000,1090000
2,Algeria,2500000,2510000,2520000,2530000,2540000,2550000,2560000,2560000,2570000,...,70400000,70500000,70500000,70600000,70700000,70700000,70700000,70700000,70700000,70700000
3,Andorra,2650,2650,2650,2650,2650,2650,2650,2650,2650,...,63000,62900,62900,62800,62700,62700,62600,62500,62500,62400
4,Angola,1570000,1570000,1570000,1570000,1570000,1570000,1570000,1570000,1570000,...,168000000,170000000,172000000,175000000,177000000,179000000,182000000,184000000,186000000,188000000


In [42]:
pib.shape

(193, 242)

In [48]:
lexp.melt(id_vars="country", var_name="year", value_name="life_expect").sort_values(by=["country", "year"])

,country,year,life_expect
0,Afghanistan,1800,28.2
187,Afghanistan,1801,28.2
374,Afghanistan,1802,28.2
561,Afghanistan,1803,28.2
748,Afghanistan,1804,28.2
...,...,...,...
55538,Zimbabwe,2096,75.1
55725,Zimbabwe,2097,75.3
55912,Zimbabwe,2098,75.4
56099,Zimbabwe,2099,75.5


In [40]:
pib_clean = pib.melt(id_vars="country", var_name="year", value_name="pib")
lexp_clean = lexp.melt(id_vars="country", var_name="year", value_name="life_expect")
murd_clean = murd.melt(id_vars="country", var_name="year", value_name="murderers")
pop_clean = pop.melt(id_vars="country", var_name="year", value_name="population")

In [49]:
pib_clean.head(1)

,country,year,pib
0,Afghanistan,1800,603


In [50]:
lexp_clean.head(1)

,country,year,life_expect
0,Afghanistan,1800,28.2


In [52]:
first_merge

,country,year,pib,life_expect
0,Afghanistan,1800,603.0,28.2
1,Albania,1800,667.0,35.4
2,Algeria,1800,715.0,28.8
3,Andorra,1800,1200.0,NaN
4,Angola,1800,618.0,27.0
...,...,...,...,...
57728,Venezuela,2100,NaN,87.3
57729,Vietnam,2100,NaN,85.3
57730,Yemen,2100,NaN,78.4
57731,Zambia,2100,NaN,78.1


In [53]:
second_merge

,country,year,pib,life_expect,murderers
0,Afghanistan,1800,603.0,28.2,NaN
1,Albania,1800,667.0,35.4,NaN
2,Algeria,1800,715.0,28.8,NaN
3,Andorra,1800,1200.0,NaN,NaN
4,Angola,1800,618.0,27.0,NaN
...,...,...,...,...,...
57728,Venezuela,2100,NaN,87.3,NaN
57729,Vietnam,2100,NaN,85.3,NaN
57730,Yemen,2100,NaN,78.4,NaN
57731,Zambia,2100,NaN,78.1,NaN


In [54]:
third_merge

,country,year,pib,life_expect,murderers,population
0,Afghanistan,1800,603.0,28.2,NaN,3280000
1,Afghanistan,1801,603.0,28.2,NaN,3280000
2,Afghanistan,1802,603.0,28.2,NaN,3280000
3,Afghanistan,1803,603.0,28.2,NaN,3280000
4,Afghanistan,1804,603.0,28.2,NaN,3280000
...,...,...,...,...,...,...
58690,Zimbabwe,2096,NaN,75.1,NaN,30900000
58691,Zimbabwe,2097,NaN,75.3,NaN,31000000
58692,Zimbabwe,2098,NaN,75.4,NaN,31000000
58693,Zimbabwe,2099,NaN,75.5,NaN,31000000


In [51]:
first_merge = pd.merge(pib_clean, lexp_clean, on=["country", "year"], how="outer")
second_merge = pd.merge(first_merge, murd_clean, on=["country", "year"], how="outer")
third_merge = pd.merge(second_merge, pop_clean, on=["country", "year"], how="outer")

third_merge.sort_values(by=["country", "year"], inplace=True)
third_merge.reset_index(drop=True, inplace=True)

df = third_merge.copy()

In [55]:
df.head()

,country,year,pib,life_expect,murderers,population
0,Afghanistan,1800,603.0,28.2,NaN,3280000
1,Afghanistan,1801,603.0,28.2,NaN,3280000
2,Afghanistan,1802,603.0,28.2,NaN,3280000
3,Afghanistan,1803,603.0,28.2,NaN,3280000
4,Afghanistan,1804,603.0,28.2,NaN,3280000


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58695 entries, 0 to 58694
Data columns (total 6 columns):
country        58695 non-null object
year           58695 non-null object
pib            46513 non-null float64
life_expect    55528 non-null float64
murderers      5049 non-null float64
population     58695 non-null int64
dtypes: float64(3), int64(1), object(2)
memory usage: 2.7+ MB


In [57]:
df.describe()

,pib,life_expect,murderers,population
count,46513.000000,55528.000000,5049.000000,5.869500e+04
mean,6428.630340,53.036046,2080.434706,2.328525e+07
std,13353.029625,21.666789,6939.441425,1.007173e+08
min,245.000000,1.010000,0.450000,6.450000e+02
25%,899.000000,32.300000,61.700000,4.220000e+05
50%,1750.000000,48.700000,260.000000,2.610000e+06
75%,4830.000000,74.200000,820.000000,1.080000e+07
max,179000.000000,94.800000,60100.000000,1.650000e+09


In [59]:
df["year"] = pd.to_numeric(df["year"]).astype("Int64")
df["pib"] = df["pib"].astype("Int64")

In [60]:
df.to_csv(folder + "country_gen_info.csv", sep="|", index=False)